#BERTopic on UNTM Dataset
In this notebook I have implemented Topic Modeling on Urdu News Data transformer based topic modelling technique BERTopic.

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.1 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039308 sha256=af2a138d6a64b0651fff164aae07c92664de044d0372af9254436ddbd5dbfeab
  Stored in dire

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling ten


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score. bertopic is used to train bertopic on our UNTM dataset with using pretrained language models

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame
We scraped 8001 urdu news that consist of 7 categories=>Business,Technology, showbiz,Health,Politics,weird and Sports. After cleaning news, total 7991 rows saved into another new column Clean_Data for experiments.



In [ ]:
#load UNTM Dataset
df = pd.read_csv('/content/drive/MyDrive/Clean_UNTM.csv', encoding='utf-8')

In [ ]:
print(len(df))
df.head()

7991


,Date,Title,News,Category,Link,Clean_Data
0,2023-04-25,انوشکا شرما اور ویرات کوہلی کی بیڈ منٹن کھیلتے...,ممبئی(اُردو پوائنٹ اخبارتازہ ترین - آن لائن۔ 2...,Sports,https://www.urdupoint.com/sports/detail-news/l...,بھارتی کرکٹ کے آئیکون ویرات کوہلی اور انکی ...
1,2023-04-25,شکست کا ملبہ کسی پر بھی نہیں ڈالوں گا،بابراعظم,راولپنڈی(اُردو پوائنٹ اخبارتازہ ترین - آن لائن...,Sports,https://www.urdupoint.com/sports/detail-news/l...,پاکستان کرکٹ ٹیم کے کپتان بابر اعظم کا کہنا ...
2,2023-04-25,آئی پی ایل:سلواوورریٹ پر کوہلی کو بھاری جرمان...,نئی دہلی (اُردو پوائنٹ اخبارتازہ ترین - آن لائ...,Sports,https://www.urdupoint.com/sports/detail-news/l...,رائل چیلنجرز بنگلور کے کپتان ویرات کوہلی کو ا...
3,2023-04-25,ٹی ٹونٹی ایک مشکل فارمیٹ ہے ،جیت کیلئے اے گیم ...,راولپنڈی (اُردو پوائنٹ اخبارتازہ ترین - آن لائ...,Sports,https://www.urdupoint.com/sports/detail-news/l...,نیوزی لینڈ سے سیریز نہ جیت پانے پر قومی ٹیم ...
4,2023-04-25,آئرلینڈ نے ٹیسٹ کرکٹ میں اپنا سب سے زیادہ ٹیس...,گال(اُردو پوائنٹ اخبارتازہ ترین - آن لائن۔ 25 ...,Sports,https://www.urdupoint.com/sports/detail-news/l...,سری لنکا اور آئرلینڈ کے درمیان جاری دو ٹیسٹ...


## Preprocessing of Data
Data was cleaned that saved into column "Clean_Data" we  removed some extra coloumns first like date,Title,Link and some other columns too. Because we do experiment only news  and all other things are extra for us.

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling.

In [ ]:
df=df.drop(columns=['Date','Title', 'Link'])

In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

df['Clean_Data'] = df['Clean_Data'].apply(remove_nonbreaking_space)

In [ ]:
documents = df['Clean_Data'].values.tolist()
print((documents[1]))

  پاکستان کرکٹ ٹیم کے کپتان بابر اعظم کا کہنا ہے کہ شکست کا ملبہ کسی پر بھی نہیں ڈالوں گا کیونکہ یہ ٹیم ورک ہوتا ہے اور ہمیشہ ایک ٹیم جیتتی ہے یا ہارتی ہے تاہم جہاں ضرورت ہے اچھا کرنے کی مزید بہتری ضرور لائیں گے۔راولپنڈی میں ٹی  میچز کی سیریز مکمل ہونے کے بعد پاکستان ٹیم کے کپتان بابر اعظم کا کہنا تھا کہ شروع میں اندازہ تھا کہ  سکور کریں گے لیکن  رنز کا اچھا سکور کیا لیکن نیوزی لینڈ کے کھلاڑی اچھا کھیلے۔ میں کبھی مطمئن نہیں ہوتا اور تربیت بھی یہی ہے کہ اچھا کرکے بھی سیکھا جاتا ہے کیونکہ بری پرفارمنس کے بعد تو ہر کوئی سیکھتا ہے۔بابر اعظم کا کہنا تھا کہ دو دن کے وقفے سے کوئی فرق نہیں پڑتا  اوور کے بعد  کا ہدف سمجھ رہے تھے لیکن یہ درست ہے کہ آخری تین اوور میں رضوان سے ہٹ نہیں لگی رضوان مزید دو رنز کر سکتے تھے لیکن نہیں کرسکے بہرحال رضوان نے بہت اچھا کھیلا کیچز ڈراپ ہوئے جو کہ نہیں ہونے چاہیے تھے۔


In [ ]:
import nltk
stopwords=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])
# stopwords['List']

stopwords_ur=[]
for li in stopwords['List']:
  stopwords_ur.append(li)
print(len(stopwords_ur))
print(stopwords_ur)

401
['کی', 'ہیں', 'ہے', 'رہا', 'رہی', 'رہے', 'تھا', 'تھے', 'تھی', 'تھى', 'میں', 'کہ', 'کے', 'ہوتے', 'کہہ', 'بنانا', 'پھر', 'لکین', 'ہوتی', 'لیتی', 'ایسی', 'ائے', 'ہوئے', 'ہوۓ ', 'مگر', 'چاہے', 'کیے', 'تاکہ', 'تم', 'آکر', 'لگا', 'ہوگیئں', 'ليے', 'رہتی', 'ہوگئی', 'انھوں', 'چاہتے', 'پاگیئں', 'آنا', 'کروا', 'رکھ', 'آتی', 'یہاں', 'جیسا', 'چکے', 'کرئے', 'دیے', 'چکا', 'ملتا', 'کبھی', 'ایسا', 'کرسکیں', 'ہوسکے', 'سکیں', 'لہذا', 'چاہئے', 'وہیں', 'اٹھایا', 'جہنوں', 'ہمارے', 'لیے', 'آرہے', 'کرگیئں', 'بنانے', 'سکتا', 'وغیرہ', 'دے', 'ہوۓ', 'رہنے', 'ہوۓ', 'کئے', 'لگے', 'لگایا', 'لائے', 'کہے', 'کرے', 'رہئں', 'آگئے', 'کئی', 'کم', 'ملی', 'جنہیں', 'ہوئیں', 'تھیں', 'ابھی', 'پاگئیں', 'آئے', 'کرا', 'دیا', 'جہاں', 'آگئیں', 'کرتی', 'رہیں', 'کرتیں', 'دیتی', 'ہوگئے', 'ديتے', 'انہں', 'ایسے', 'رکھتے', 'رہتے', 'رکھی', 'کیں', 'كیا', 'وه', 'جنہيں', 'کروائی', 'دینا', 'جسے', 'جاتی', 'اسکی', 'ملے', 'کرگئى', 'جن', 'آپکو', 'جنہوں', 'دیئے', 'والی', 'جبکہ', 'دیگر', 'آپکا', 'رکھنے', 'انہىں', 'کيے', 'یعنی', 'كيے', 'بننے', 'ج

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". we also performed experiment on 2 another pretained multilingual model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(documents, show_progress_bar=True)
print(embeddings)

.gitattributes:   0%|          | 0.00/968 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/250 [00:00<?, ?it/s]

[[ 0.07987808 -0.07742516 -0.08378563 ... -0.043457   -0.11484845
  -0.07181289]
 [ 0.09646066 -0.04020533 -0.04377843 ...  0.00730114  0.09356619
   0.05510345]
 [ 0.00332669  0.09647436 -0.17899874 ... -0.15398908  0.01223139
  -0.0459842 ]
 ...
 [-0.05227447  0.06783362 -0.2360085  ... -0.18109454 -0.0346763
   0.08866747]
 [ 0.1034833   0.31677362  0.00757494 ...  0.09555458  0.02051694
   0.20520177]
 [-0.09281214  0.19791466 -0.18801452 ... -0.22458135 -0.11964664
   0.11853428]]


In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stopwords_ur)


In [ ]:
#ClassTfidf for top words
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(bm25_weighting=True, reduce_frequent_words=True)

In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_neighbors=15, n_components=2, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=7, random_state=42)

In [ ]:
# from sklearn.cluster import AgglomerativeClustering

# cluster_model = AgglomerativeClustering(n_clusters=7)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True, vectorizer_model=vectorizer_model, top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model,ctfidf_model=ctfidf_model, verbose=True)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(documents,embeddings)

2023-12-29 02:03:43,738 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-29 02:04:35,686 - BERTopic - Dimensionality - Completed ✓
2023-12-29 02:04:35,689 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-29 02:04:35,901 - BERTopic - Cluster - Completed ✓
2023-12-29 02:04:35,918 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-29 02:04:40,298 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[1, 5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 0, 5, 5, 0, 5, 2, 5, 5, 5, 5, 5, 2, 0, 5, 1, 3, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 1, 5, 5, 1, 5, 2, 5, 5, 5, 5, 1, 5, 2, 5, 5, 5, 1, 2, 5, 5, 5, 5, 5, 0, 0, 5, 5, 5, 5, 0, 5, 1, 1, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 0, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 0, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 0, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 0, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 0, 5, 5, 5, 1, 5, 5, 1, 5, 0, 3, 1, 3, 5, 5, 5, 5, 2, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
4,0,1626
0,1,1290
2,2,1263
3,3,1062
5,4,928
1,5,916
6,6,906


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('پولیو', 0.32556417645265573), ('ڈاکٹر', 0.28346584935255104), ('مہم', 0.27962763042833466), ('بتایا', 0.26569437238832627), ('صحت', 0.263160333492338), ('پولیس', 0.2617581696427421), ('بچوں', 0.25367031099692144), ('ہسپتال', 0.2520775128885678), ('ڈپٹی', 0.24692273817229313), ('گھر', 0.24413312417593774)], 1: [('فلم', 0.5110293656863505), ('اداکارہ', 0.45045858411989886), ('اداکار', 0.39146381419628784), ('شادی', 0.3798540540962347), ('بالی', 0.3689936595164652), ('میڈیا', 0.3616513954434571), ('خان', 0.35952876565772146), ('بھارتی', 0.3573395841923862), ('ووڈ', 0.355021952119844), ('شاہ', 0.31643697898361767)], 2: [('عمران', 0.41946958058136025), ('خان', 0.3951826739137994), ('تحریک', 0.37235637688902984), ('حکومت', 0.3575358363348081), ('لیگ', 0.34775534250271556), ('وزیراعظم', 0.34555358481877435), ('الیکشن', 0.33987801664158), ('شریف', 0.33888412291874614), ('انصاف', 0.3332367348446401), ('نواز', 0.3115696480891967)], 3: [('روپے', 0.3797454804012403), ('مالی', 0.376280950030

In [ ]:
#shows which documents assign to specific topic
topic_model.get_representative_docs(topic=None)

{0: ['  پولیو ایک مہلک مرض ہے جس کا ہر صورت خاتمہ ہونا چاہیے اور یہ اس وقت ممکن ہے جب حکومتی اداروں کے سربراہ ایک دوسرے کے ساتھ تعاون کریں اور مل جل کر کام کریں اور یہ جانتے ہوئے کہ یہ ایک قومی کاز ہے۔ اس سلسلے میں اپنا کردار ادا کریں۔ان خیالات کا اظہار کمشنر لاڑکانہ گھنور خان لغاری نے اپنے دفتر میں ڈویژنل ہیلتھ مینجمنٹپولیو سے متعلق ڈویژنل ٹاسک فورس کے ایک اجلاس کی صدارت کرتے ہوئے کیا۔ اجلاس میں ڈویژن کے تمام ڈپٹی کمشنرز ایڈیشنل ڈپٹی کمشنرز ڈسٹرکٹ ہیلتھ آفیسرز اور ڈبلیو ایچ او کے نمائندوں نے شرکت کی جبکہ ڈپٹی کمشنر لاڑکانہ طارق منظور اجلاس میں موجود تھے۔ اس موقع پر انہوں نے کہا کہ پولیو کا مقابلہ کرنا اور اس سے چھٹکارا حاصل کرنا ہم سب کی مشترکہ ذمہ داری ہے۔ انہوں نے محکمہ صحت کے افسران کو ہدایت کی کہ بچوں کو پولیو سے بچاو ¿ کے قطرے پلانے کو ہر صورت یقینی بنایا جائے۔پولیو ٹیموں کی نگرانی کو یقینی بنایا جائے اگر آپ کو پولیو کے خاتمے کے لیے کسی اور محکمے کی مدد کی ضرورت ہے تو اس مسئلے کو حل کرنے میں مدد کے لیے ضلع کے ڈی سیز سے رابطہ کریں اور محکمہ صحت کا عملہ اپنی ماہانہ رپورٹس ڈی پی سی 

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,1626,0_پولیو_ڈاکٹر_مہم_بتایا,"[پولیو, ڈاکٹر, مہم, بتایا, صحت, پولیس, بچوں, ہ...",[ پولیو ایک مہلک مرض ہے جس کا ہر صورت خاتمہ ہ...
1,1,1290,1_فلم_اداکارہ_اداکار_شادی,"[فلم, اداکارہ, اداکار, شادی, بالی, میڈیا, خان,...",[ بھارتی ورسٹائل اداکار نواز الدین صدیقی نے ا...
2,2,1263,2_عمران_خان_تحریک_حکومت,"[عمران, خان, تحریک, حکومت, لیگ, وزیراعظم, الیک...",[حکومت اور عمران خان کی زیر قیادت پی ٹی آئی ک...
3,3,1062,3_روپے_مالی_ملین_سال,"[روپے, مالی, ملین, سال, ڈالر, فیصد, بینک, ارب,...",[اسلام آباد ملک کے صف ِ اول کے ٹیلی کام اور آ...
4,4,928,4_فون_صارفین_ویوو_موبائل,"[فون, صارفین, ویوو, موبائل, بک, فیس, کمپنی, ای...",[ ویوو نے سمارٹ فون کی دنیا میں سیلفی فوٹوگراف...
5,5,916,5_ٹیم_میچ_کرکٹ_رنز,"[ٹیم, میچ, کرکٹ, رنز, ٹورنامنٹ, سیریز, کھلاڑیو...",[ مشیر کھیل پنجاب وہاب ریاض نے کہا ہے کہ نگران...
6,6,906,6_کورونا_وائرس_کیسز_مریض,"[کورونا, وائرس, کیسز, مریض, ہزار, ویکسین, افرا...",[ صوبائی سیکرٹری محکمہ پرائمری اینڈسیکنڈری ہیل...


In [ ]:
topic_model.get_document_info(documents)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,بھارتی کرکٹ کے آئیکون ویرات کوہلی اور انکی ...,1,1_فلم_اداکارہ_اداکار_شادی,"[فلم, اداکارہ, اداکار, شادی, بالی, میڈیا, خان,...",[ بھارتی ورسٹائل اداکار نواز الدین صدیقی نے ا...,فلم - اداکارہ - اداکار - شادی - بالی - میڈیا -...,False
1,پاکستان کرکٹ ٹیم کے کپتان بابر اعظم کا کہنا ...,5,5_ٹیم_میچ_کرکٹ_رنز,"[ٹیم, میچ, کرکٹ, رنز, ٹورنامنٹ, سیریز, کھلاڑیو...",[ مشیر کھیل پنجاب وہاب ریاض نے کہا ہے کہ نگران...,ٹیم - میچ - کرکٹ - رنز - ٹورنامنٹ - سیریز - کھ...,False
2,رائل چیلنجرز بنگلور کے کپتان ویرات کوہلی کو ا...,5,5_ٹیم_میچ_کرکٹ_رنز,"[ٹیم, میچ, کرکٹ, رنز, ٹورنامنٹ, سیریز, کھلاڑیو...",[ مشیر کھیل پنجاب وہاب ریاض نے کہا ہے کہ نگران...,ٹیم - میچ - کرکٹ - رنز - ٹورنامنٹ - سیریز - کھ...,False
3,نیوزی لینڈ سے سیریز نہ جیت پانے پر قومی ٹیم ...,5,5_ٹیم_میچ_کرکٹ_رنز,"[ٹیم, میچ, کرکٹ, رنز, ٹورنامنٹ, سیریز, کھلاڑیو...",[ مشیر کھیل پنجاب وہاب ریاض نے کہا ہے کہ نگران...,ٹیم - میچ - کرکٹ - رنز - ٹورنامنٹ - سیریز - کھ...,False
4,سری لنکا اور آئرلینڈ کے درمیان جاری دو ٹیسٹ...,5,5_ٹیم_میچ_کرکٹ_رنز,"[ٹیم, میچ, کرکٹ, رنز, ٹورنامنٹ, سیریز, کھلاڑیو...",[ مشیر کھیل پنجاب وہاب ریاض نے کہا ہے کہ نگران...,ٹیم - میچ - کرکٹ - رنز - ٹورنامنٹ - سیریز - کھ...,False
...,...,...,...,...,...,...,...
7986,پشاور کے لیڈی ریڈنگ ہسپتال میں کورونا وائرس ...,6,6_کورونا_وائرس_کیسز_مریض,"[کورونا, وائرس, کیسز, مریض, ہزار, ویکسین, افرا...",[ صوبائی سیکرٹری محکمہ پرائمری اینڈسیکنڈری ہیل...,کورونا - وائرس - کیسز - مریض - ہزار - ویکسین -...,False
7987,لبنان کی وزارت صحت نے کووِڈکی اومیکرون قسم کے...,6,6_کورونا_وائرس_کیسز_مریض,"[کورونا, وائرس, کیسز, مریض, ہزار, ویکسین, افرا...",[ صوبائی سیکرٹری محکمہ پرائمری اینڈسیکنڈری ہیل...,کورونا - وائرس - کیسز - مریض - ہزار - ویکسین -...,False
7988,عالمی ادارہ صحت کے ویکسین ایڈوائزری پینل نے ص...,6,6_کورونا_وائرس_کیسز_مریض,"[کورونا, وائرس, کیسز, مریض, ہزار, ویکسین, افرا...",[ صوبائی سیکرٹری محکمہ پرائمری اینڈسیکنڈری ہیل...,کورونا - وائرس - کیسز - مریض - ہزار - ویکسین -...,False
7989,ڈپٹی کمشنر حامد محمود ملہی کے زیر صدارت ضلعی...,0,0_پولیو_ڈاکٹر_مہم_بتایا,"[پولیو, ڈاکٹر, مہم, بتایا, صحت, پولیس, بچوں, ہ...",[ پولیو ایک مہلک مرض ہے جس کا ہر صورت خاتمہ ہ...,پولیو - ڈاکٹر - مہم - بتایا - صحت - پولیس - بچ...,False


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(documents, window=3, min_similarity=0.01)

100%|██████████| 8/8 [00:22<00:00,  2.87s/it]


In [ ]:
print(topic_distr)

[[0.11684288 0.28642573 0.11363627 ... 0.13931353 0.1938752  0.08840713]
 [0.11600488 0.13558588 0.13482894 ... 0.11332278 0.31975295 0.09039962]
 [0.08312012 0.16099738 0.10546112 ... 0.11278953 0.30759097 0.1206529 ]
 ...
 [0.15395804 0.07299337 0.09715051 ... 0.14086887 0.08882416 0.29775716]
 [0.24288742 0.08436421 0.13238186 ... 0.10801591 0.11215143 0.20306891]
 [0.16101096 0.09399016 0.12720495 ... 0.14063873 0.10744079 0.2358624 ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distributio(UNTM)")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
texts = [[word for word in str(document).split() if word not in stopwords_ur] for document in documents]
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['پولیو', 'ڈاکٹر', 'مہم', 'بتایا', 'صحت', 'پولیس', 'بچوں', 'ہسپتال', 'ڈپٹی', 'گھر'], ['فلم', 'اداکارہ', 'اداکار', 'شادی', 'بالی', 'میڈیا', 'خان', 'بھارتی', 'ووڈ', 'شاہ'], ['عمران', 'خان', 'تحریک', 'حکومت', 'لیگ', 'وزیراعظم', 'الیکشن', 'شریف', 'انصاف', 'نواز'], ['روپے', 'مالی', 'ملین', 'سال', 'ڈالر', 'فیصد', 'بینک', 'ارب', 'کمی', 'فروری'], ['فون', 'صارفین', 'ویوو', 'موبائل', 'بک', 'فیس', 'کمپنی', 'ایپ', 'سمارٹ', 'ساتھ'], ['ٹیم', 'میچ', 'کرکٹ', 'رنز', 'ٹورنامنٹ', 'سیریز', 'کھلاڑیوں', 'بال', 'سپورٹس', 'نیوزی'], ['کورونا', 'وائرس', 'کیسز', 'مریض', 'ہزار', 'ویکسین', 'افراد', 'گھنٹوں', 'صحت', 'مریضوں']]


In [ ]:
# compute Coherence Score CV and NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCoherence Score: ', coherence)


Coherence Score:  0.79


**Diversity Score**

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 1.0


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores (UNTM)")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2023-12-29 02:21:06,661 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,1626,0_پولیو_ڈاکٹر_مہم_بتایا,"[پولیو, ڈاکٹر, مہم, بتایا, صحت, پولیس, بچوں, ہ...",[ پولیو ایک مہلک مرض ہے جس کا ہر صورت خاتمہ ہ...
1,1,1290,1_فلم_اداکارہ_اداکار_شادی,"[فلم, اداکارہ, اداکار, شادی, بالی, میڈیا, خان,...",[ بھارتی ورسٹائل اداکار نواز الدین صدیقی نے ا...
2,2,1263,2_عمران_خان_تحریک_حکومت,"[عمران, خان, تحریک, حکومت, لیگ, وزیراعظم, الیک...",[حکومت اور عمران خان کی زیر قیادت پی ٹی آئی ک...
3,3,1062,3_روپے_مالی_ملین_سال,"[روپے, مالی, ملین, سال, ڈالر, فیصد, بینک, ارب,...",[اسلام آباد ملک کے صف ِ اول کے ٹیلی کام اور آ...
4,4,928,4_فون_صارفین_ویوو_موبائل,"[فون, صارفین, ویوو, موبائل, بک, فیس, کمپنی, ای...",[ ویوو نے سمارٹ فون کی دنیا میں سیلفی فوٹوگراف...
5,5,916,5_ٹیم_میچ_کرکٹ_رنز,"[ٹیم, میچ, کرکٹ, رنز, ٹورنامنٹ, سیریز, کھلاڑیو...",[ مشیر کھیل پنجاب وہاب ریاض نے کہا ہے کہ نگران...
6,6,906,6_کورونا_وائرس_کیسز_مریض,"[کورونا, وائرس, کیسز, مریض, ہزار, ویکسین, افرا...",[ صوبائی سیکرٹری محکمہ پرائمری اینڈسیکنڈری ہیل...
